# View Motion in 3D Animation

## Required Packages

In [1]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

## Customized Tools

In [2]:
'''
Compute triangle functions based on angle.
The angle is in the unit of degrees.
'''
def sin(x):
    '''
    Compute sin of [x] in the unit of degrees.
    
    Args:
    - @x: The angle value in degrees.
    '''
    return np.sin(x / 180 * np.pi)


def cos(x):
    '''
    Compute cos of [x] in the unit of degrees.
    
    Args:
    - @x: The angle value in degrees.
    '''
    return np.cos(x / 180 * np.pi)


def rotate(rot):
    '''
    Compute rotate matrix for [rot].
    The matrix is designed to be right-multipled to the vector to perform the rotation.
    
    Args:
    - @rot: The rotation's 3 params, the unit is degree.
    '''
    rx, ry, rz = rot

    mz = np.array([[cos(rz), sin(rz), 0],
                   [-sin(rz), cos(rz), 0],
                   [0, 0, 1]])

    my = np.array([[cos(ry), 0, -sin(ry)],
                   [0, 1, 0],
                   [sin(ry), 0, cos(ry)]])

    mx = np.array([[1, 0, 0],
                   [0, cos(rx), sin(rx)],
                   [0, -sin(rx), cos(rx)]])

    mat = np.matmul(np.matmul(mz, my), mx)

    return mat


'''
Generate rgb hex string for colors in colormap of 'cool'
'''
def rgb(i, n=20):
    '''
    Generate color in hex format, like "#FFFFFF".
    
    Args:
    - @i: The index of the color;
    - @n: The max of the color index, default by 20.
    '''
    c = plt.cm.cool(i/n)
    return matplotlib.colors.to_hex(c)

In [3]:
class Trace(object):
    ''' Trace Motion and Manage the Positions of the Rod '''
    
    def __init__(self, vec, pos=[0, 0, 0], rot=[0, 0, 0], color='black'):
        '''
        Init the Trace.
        
        The purpose of the trace is tracing the rigid transformation of the rod.
        The tracing is saved in [self.df],
        the columns are ['x', 'y', 'z', 'color', 'step'],
        each trace has two rows:
          1. The start end of the rod;
          2. The stop end of the rod.
        
        The [vec] refers the origin rod position (without rigid transformation),
        and the [pos] / [rot] params refer the 6-value rigid transformation.
        
        Args:
        - @vec: The initial vector of the rod;
        - @pos: The current position of the rod, default by [0, 0, 0], unit is unknown;
        - @rot: The rotation of the rod, default by [0, 0, 0], unit is degree;
        - @color: The color of the rod, default by 'black'.
        '''
        
        self.vec = np.array(vec).astype(np.float)
        self.pos = np.array(pos).astype(np.float)
        self.rot = np.array(rot).astype(np.float)

        self.step = 0

        rod = self._place()
        rod['color'] = [color for _ in range(len(rod))]
        rod['step'] = 0

        self.df = rod

    def _place(self):
        '''
        Compute the position of the rod with rigid transformation
        '''
        
        new_vec = np.matmul(self.vec, rotate(self.rot))
        p2 = self.pos + new_vec

        rod = pd.DataFrame(np.array([self.pos, p2]))
        rod.columns = ['x', 'y', 'z']

        return rod

    def add(self, pos_diff=[0, 0, 0], rot_diff=[0, 0, 0], color='black'):
        '''
        Add new trace of the rod,
        the new trace will be added to the [self.df].
        
        Args:
        - @pos_diff: The latest change of the position's 3 params, default by [0, 0, 0];
        - @rot_diff: The latest change of the rotation's 3 params, default by [0, 0, 0];
        - @color: The color of the rod, default by 'black'.
        '''
        
        self.pos += np.array(pos_diff).astype(np.float)
        self.rot += np.array(rot_diff).astype(np.float)
        self.step += 1

        rod = self._place()
        rod['color'] = [color for _ in range(len(rod))]
        rod['step'] = self.step

        self.df = pd.concat([self.df, rod], axis=0)

In [4]:
'''
Plot Animation
'''

length = 10
vec = [-length, 0, 0]

def plot_animation(md, vec=vec):
    e, dd = md
    trace = Trace(vec, color=rgb(0))

    for i, d in enumerate(dd):
        trace.add(pos_diff=d[:3], rot_diff=d[3:], color=rgb(i))

    trace.df

    range_x=(trace.df['x'].min(), trace.df['x'].max())
    range_y=(trace.df['y'].min(), trace.df['y'].max())
    range_z=(trace.df['z'].min(), trace.df['z'].max())

    df1 = trace.df.copy()
    df1['line_group'] = 'a'

    # Add Grids
    dfx = trace.df.copy()
    dfx['line_group'] = 'g'

    dfy = trace.df.copy()
    dfy['line_group'] = 'g'

    dfz = trace.df.copy()
    dfz['line_group'] = 'g'

    # x-axis
    xyz = dfx[['x', 'y', 'z']].values
    for j in range(len(xyz)):
        if j % 2 == 0:
            xyz[j] = [range_x[0], range_y[1], range_z[1]]
        if j % 2 == 1:
            xyz[j] = [range_x[1], range_y[1], range_z[1]]
    dfx[['x', 'y', 'z']] = xyz

    # y-axis
    xyz = dfy[['x', 'y', 'z']].values
    for j in range(len(xyz)):
        if j % 2 == 0:
            xyz[j] = [range_x[0], range_y[1], range_z[1]]
        if j % 2 == 1:
            xyz[j] = [range_x[0], range_y[0], range_z[1]]
    dfy[['x', 'y', 'z']] = xyz

    # z-axis
    xyz = dfz[['x', 'y', 'z']].values
    for j in range(len(xyz)):
        if j % 2 == 0:
            xyz[j] = [range_x[0], range_y[1], range_z[1]]
        if j % 2 == 1:
            xyz[j] = [range_x[0], range_y[1], range_z[0]]
    dfz[['x', 'y', 'z']] = xyz

    df = pd.concat([df1, dfx, dfy, dfz], axis=0)

    kwargs = dict(
        width=800,
        height=800,
        title=e
    )

    fig = px.line_3d(df, x='x', y='y', z='z', line_group='line_group',
                     color='color', animation_frame='step', **kwargs)

    for f in fig.frames:
        f.data[0].name = 'x'
        # f.data[1].name = '--'

    for j, frame in enumerate(fig.frames):
        frame['data'][0]['line']['color'] = df.iloc[j*2]['color']
        frame['data'][0]['line']['width'] = 5
        frame['data'][1]['line']['color'] = 'black'

    fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 20
    
    return fig, trace.df

## Load Data

In [5]:
''' Parameters '''
data_folder = r'H:\Sync\MotionData\data'

event_name = ['0-SLW', '1-MLW', '2-FLW', '3-RD',
              '4-SD', '5-sit', '6-stand', '7-RA', '8-SA']

channel_names = ['ax', 'ay', 'az', 'gx', 'gy', 'gz']

'''
The data_3d shape is (31625, 20, 6),
                 samples x times x channels
The event shape is (31625,)
'''
data_3d = np.load(os.path.join(data_folder, 'data3.npy'))
event = np.load(os.path.join(data_folder, 'event.npy'))
print(data_3d.shape, event.shape)

# data_3d[:, :, 2] = 0

'''
The mean_data is the averaged data of all the 9 events.
'''
mean_data = []
for e in np.unique(event):
    en = event_name[int(e-1)]
    mean_data.append([en, np.mean(data_3d[event == e], axis=0)])
    

'''
Report
'''
print(f'The mean_data Report:')
for md in mean_data:
    print('\t', md[0], '\t', md[1].shape)

(31625, 20, 6) (31625,)
The mean_data Report:
	 0-SLW 	 (20, 6)
	 1-MLW 	 (20, 6)
	 2-FLW 	 (20, 6)
	 3-RD 	 (20, 6)
	 4-SD 	 (20, 6)
	 5-sit 	 (20, 6)
	 6-stand 	 (20, 6)
	 7-RA 	 (20, 6)
	 8-SA 	 (20, 6)


## Plot Animation

In [11]:
md = mean_data[0]
fig, df = plot_animation(md)
df

,x,y,z,color,step
0,0.000000,0.000000,0.000000,#00ffff,0
1,-10.000000,0.000000,0.000000,#00ffff,0
0,-0.126842,-0.053120,-1.093271,#00ffff,1
1,-10.002347,-1.261171,-0.085794,#00ffff,1
0,-0.249742,-0.109073,-2.188722,#0cf3ff,2
1,-9.749686,-2.793700,-0.593774,#0cf3ff,2
0,-0.368764,-0.166661,-3.285113,#19e6ff,3
1,-9.246518,-4.430747,-1.552183,#19e6ff,3
0,-0.486211,-0.224295,-4.385412,#26d9ff,4
1,-8.583352,-5.950914,-3.103957,#26d9ff,4


In [12]:
camera = dict(
    up=dict(x=1, y=0, z=0),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0, y=-2, z=0)
)

fig.update_layout(scene_camera=camera)
fig.show()

In [8]:
html = open('template_motion_animation.html').read()
include_plotlyjs = True

for md in mean_data:
    e, _ = md
    fig, _ = plot_animation(md)
    fig.update_layout(scene_camera=camera)
    # fig.show()
    html = html.replace(f'{{{{event {e}}}}}',
                        fig.to_html(full_html=False,
                                    include_plotlyjs=include_plotlyjs))
    print(len(html))
    include_plotlyjs = False

with open('motion_animation_tmp.html', 'w') as f:
    f.write(html)

3514410
3546957
3581382
3615503
3651493
3685651
3719822
3754258
3790342


In [9]:
html = open('template_motion_animation.html').read()
include_plotlyjs = True

for md in mean_data:
    e, dd = md

    trace = Trace(vec, color=rgb(0))

    for i, d in enumerate(dd):
        trace.add(pos_diff=d[:3], rot_diff=d[3:], color=rgb(i))

    data = []
    layout = dict(
        title=e,
    )

    for i in trace.df['step'].unique():
        df = trace.df.query(f'step=={i}')

        d = go.Scatter3d(
            x=df['x'],
            y=df['y'],
            z=df['z'],
            marker=dict(
                size=2,
                color=df['color'],
            ),
            line=dict(
                color=df['color']
            )
        )

        data.append(d)

    fig = go.Figure(data=data, layout=layout)
    fig.update_layout(scene_camera=camera)
    # fig.show()
    
    html = html.replace(f'{{{{event {e}}}}}',
                        fig.to_html(full_html=False,
                                    include_plotlyjs=include_plotlyjs))
    print(len(html))
    include_plotlyjs = False

with open('motion_trace_tmp.html', 'w') as f:
    f.write(html)

3492447
3506133
3519805
3533481
3547156
3560869
3574595
3588300
3601981
